<a href="https://colab.research.google.com/github/sungjindo/black_coal_price/blob/main/blackcoal_date.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 기본 라이브러리
import pandas as pd
import numpy as np
import os

# 시각화 라이브러리
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib notebook
%matplotlib inline

# 통계 및 기계학습 관련 라이브러리
from scipy import stats    # 통계 관련 라이브러리
from sklearn.model_selection import train_test_split, RandomizedSearchCV    # 데이터 셋 분류 라이브러리
from sklearn.utils import resample    # 데이터 업샘플링&다운샘플링 라이브러리
from imblearn.over_sampling import RandomOverSampler
from imblearn.combine import SMOTEENN    # 복합샘플링 라이브러리
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler    # 데이터 스케일링 라이브러리
from statsmodels.stats.outliers_influence import variance_inflation_factor    # 변수 간 다중공선성 확인(VIF) 라이브러리
from sklearn.metrics import (accuracy_score, confusion_matrix, classification_report, roc_auc_score, roc_curve, auc, plot_confusion_matrix, plot_roc_curve)    # 모델 정확도 검증 라이브러리
from sklearn.model_selection import cross_val_score    # validation  검증 라이브러리
from sklearn.linear_model import LogisticRegression    # 로지스틱 회귀 라이브러리
from sklearn.linear_model import Lasso    # 라쏘 라이브러리
from sklearn.linear_model import Ridge    # 릿지 라이브러리
import statsmodels.api as sm    # 통계분석 관련 라이브러리
from xgboost import XGBClassifier    # XGBoost 라이브러리
from sklearn.ensemble import RandomForestClassifier    # 랜덤포레스트 라이브러리
import warnings    # 경고문구 지우기

from google.colab import drive
drive.mount('/content/gdrive')

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Mounted at /content/gdrive


#DATA

In [ ]:
df_up = df1.resample('W-SUN').ffill()
df_up

# 기간 참고 https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects
# 개념 참고 https://datascienceschool.net/01%20python/04.08%20%EC%8B%9C%EA%B3%84%EC%97%B4%20%EC%9E%90%EB%A3%8C%20%EB%8B%A4%EB%A3%A8%EA%B8%B0.html


,KPPI,ACPI,KCPI,APPI,APPI2
date,,,,,
2000-01-02,3.1,168.800,1.9,NaN,NaN
2000-01-09,3.1,168.800,1.9,NaN,NaN
2000-01-16,3.1,168.800,1.9,NaN,NaN
2000-01-23,3.1,168.800,1.9,NaN,NaN
2000-01-30,3.1,168.800,1.9,NaN,NaN
...,...,...,...,...,...
2020-11-08,-0.5,260.229,0.6,0.003,124.674642
2020-11-15,-0.5,260.229,0.6,0.003,124.674642
2020-11-22,-0.5,260.229,0.6,0.003,124.674642


# data_combine

In [11]:
df_0 = pd.read_csv("/content/gdrive/My Drive/Black_coal/data/최종데이터.csv", encoding = 'utf-8-sig')
df_0['date']= pd.to_datetime(df_0['date'])

df_0 = df_0.set_index('date')

df_0


,Coal_price,iron,Brent_fut,copper_fut,WTI_fut,WTI,LNG_fut
date,,,,,,,
2008-12-08,70.45,NaN,43.42,"3,294.00",43.71,44.01,54.00
2008-12-09,70.50,NaN,41.53,"3,175.50",42.07,42.40,53.72
2008-12-10,71.07,NaN,42.40,"3,275.50",43.52,44.09,56.81
2008-12-11,75.88,NaN,47.39,"3,287.00",47.98,47.31,62.29
2008-12-12,74.04,NaN,46.41,"3,143.50",46.28,46.37,59.57
...,...,...,...,...,...,...,...
2022-05-20,417.25,133.66,112.55,"9,422.00",113.23,112.04,147.96
2022-05-21,417.25,133.66,112.55,"9,422.00",113.23,112.04,147.96
2022-05-22,417.25,133.66,112.55,"9,422.00",113.23,111.89,147.96


In [7]:
df_1 = pd.read_csv("/content/gdrive/My Drive/Black_coal/data/bdi.csv", encoding = 'utf-8-sig')
df_1['date']= pd.to_datetime(df_1['Date'])
df_1 = df_1[['date','Price']]
df_1 = df_1.set_index('date')
df_1 = df_1.rename(columns={'Price':'bdi'})
df_1


,bdi
date,
2000-01-03,1318
2000-01-04,1320
2000-01-05,1329
2000-01-06,1351
2000-01-07,1368
...,...
2022-05-19,3289
2022-05-20,3344
2022-05-21,3344


In [9]:
df_2 = pd.read_csv("/content/gdrive/My Drive/Black_coal/data/finance data.csv", encoding = 'utf-8-sig')
df_2 = df_2.rename(columns={'Unnamed: 0':'date'})
df_2['date']= pd.to_datetime(df_2['date'])

df_2 = df_2.set_index('date')
df_2

,USD/AUD(호주),USD/CAD(캐나다),USD/COP(콜롬비아),US 달러지수,USD/IDR(인도네시아),USD/RUB(러시아),USD/ZAR(자메이카),상하이종합지수,인도지수,브라질지수,...,호주지수,캐나다지수,베트남GDP,인도GDP,한국GDP,일본GDP,중국GDP,미국GDP,EUGDP,세계GDP
date,,,,,,,,,,,,,,,,,,,,,
2009-01-01,1.418400,1.22180,2284.100098,82.150002,11142.000000,29.142000,9.47760,1820.805054,9647.309570,37550.0,...,3722.300049,8987.700195,1.060147e+11,1.341887e+12,9.439419e+11,5.289493e+12,5.101703e+12,1.444893e+13,1.476378e+13,6.078101e+13
2009-01-02,1.409800,1.20940,2221.800049,82.805000,10991.000000,28.697001,9.28000,1820.805054,9958.219727,40244.0,...,3713.800049,9234.099609,1.060147e+11,1.341887e+12,9.439419e+11,5.289493e+12,5.101703e+12,1.444893e+13,1.476378e+13,6.078101e+13
2009-01-03,1.409800,1.20940,2221.800049,82.805000,10991.000000,28.697001,9.28000,1820.805054,9958.219727,40244.0,...,3713.800049,9234.099609,1.060147e+11,1.341887e+12,9.439419e+11,5.289493e+12,5.101703e+12,1.444893e+13,1.476378e+13,6.078101e+13
2009-01-04,1.409800,1.20940,2221.800049,82.805000,10991.000000,28.697001,9.28000,1820.805054,9958.219727,40244.0,...,3713.800049,9234.099609,1.060147e+11,1.341887e+12,9.439419e+11,5.289493e+12,5.101703e+12,1.444893e+13,1.476378e+13,6.078101e+13
2009-01-05,1.400000,1.18800,2219.899902,83.574997,10929.000000,28.943001,9.26630,1880.715942,10275.599609,41519.0,...,3687.000000,9285.500000,1.060147e+11,1.341887e+12,9.439419e+11,5.289493e+12,5.101703e+12,1.444893e+13,1.476378e+13,6.078101e+13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-27,1.314613,1.28482,3503.250000,90.250000,14148.643555,74.029999,14.53520,3396.562988,46973.539062,117857.0,...,6664.799805,17623.900391,2.711584e+11,2.660245e+12,1.637896e+12,5.057759e+12,1.472273e+13,2.095303e+13,1.529193e+13,8.474698e+13
2020-12-28,1.313730,1.28396,3513.250000,90.275002,13911.143555,73.954903,14.60990,3397.284912,47353.750000,119051.0,...,6664.799805,17623.900391,2.711584e+11,2.660245e+12,1.637896e+12,5.057759e+12,1.472273e+13,2.095303e+13,1.529193e+13,8.474698e+13
2020-12-29,1.317710,1.28400,3498.250000,89.917999,14126.120117,73.930496,14.62291,3379.035889,47613.078125,119475.0,...,6700.299805,17543.400391,2.711584e+11,2.660245e+12,1.637896e+12,5.057759e+12,1.472273e+13,2.095303e+13,1.529193e+13,8.474698e+13


In [33]:
df_11 = df_2['세계GDP']
df_12 = df_2.iloc[:,:-1]
df_13 = pd.merge(df_11, df_12, left_index = True ,right_index = True, how='left')
df_13

,세계GDP,USD/AUD(호주),USD/CAD(캐나다),USD/COP(콜롬비아),US 달러지수,USD/IDR(인도네시아),USD/RUB(러시아),USD/ZAR(자메이카),상하이종합지수,인도지수,...,유로스톡스50지수,호주지수,캐나다지수,베트남GDP,인도GDP,한국GDP,일본GDP,중국GDP,미국GDP,EUGDP
date,,,,,,,,,,,,,,,,,,,,,
2009-01-01,6.078101e+13,1.418400,1.22180,2284.100098,82.150002,11142.000000,29.142000,9.47760,1820.805054,9647.309570,...,2451.479980,3722.300049,8987.700195,1.060147e+11,1.341887e+12,9.439419e+11,5.289493e+12,5.101703e+12,1.444893e+13,1.476378e+13
2009-01-02,6.078101e+13,1.409800,1.20940,2221.800049,82.805000,10991.000000,28.697001,9.28000,1820.805054,9958.219727,...,2451.479980,3713.800049,9234.099609,1.060147e+11,1.341887e+12,9.439419e+11,5.289493e+12,5.101703e+12,1.444893e+13,1.476378e+13
2009-01-03,6.078101e+13,1.409800,1.20940,2221.800049,82.805000,10991.000000,28.697001,9.28000,1820.805054,9958.219727,...,2451.479980,3713.800049,9234.099609,1.060147e+11,1.341887e+12,9.439419e+11,5.289493e+12,5.101703e+12,1.444893e+13,1.476378e+13
2009-01-04,6.078101e+13,1.409800,1.20940,2221.800049,82.805000,10991.000000,28.697001,9.28000,1820.805054,9958.219727,...,2451.479980,3713.800049,9234.099609,1.060147e+11,1.341887e+12,9.439419e+11,5.289493e+12,5.101703e+12,1.444893e+13,1.476378e+13
2009-01-05,6.078101e+13,1.400000,1.18800,2219.899902,83.574997,10929.000000,28.943001,9.26630,1880.715942,10275.599609,...,2553.409912,3687.000000,9285.500000,1.060147e+11,1.341887e+12,9.439419e+11,5.289493e+12,5.101703e+12,1.444893e+13,1.476378e+13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-27,8.474698e+13,1.314613,1.28482,3503.250000,90.250000,14148.643555,74.029999,14.53520,3396.562988,46973.539062,...,3539.260010,6664.799805,17623.900391,2.711584e+11,2.660245e+12,1.637896e+12,5.057759e+12,1.472273e+13,2.095303e+13,1.529193e+13
2020-12-28,8.474698e+13,1.313730,1.28396,3513.250000,90.275002,13911.143555,73.954903,14.60990,3397.284912,47353.750000,...,3575.409912,6664.799805,17623.900391,2.711584e+11,2.660245e+12,1.637896e+12,5.057759e+12,1.472273e+13,2.095303e+13,1.529193e+13
2020-12-29,8.474698e+13,1.317710,1.28400,3498.250000,89.917999,14126.120117,73.930496,14.62291,3379.035889,47613.078125,...,3581.370117,6700.299805,17543.400391,2.711584e+11,2.660245e+12,1.637896e+12,5.057759e+12,1.472273e+13,2.095303e+13,1.529193e+13


In [12]:
df_left = pd.merge(df_0, df_1, left_index = True ,right_index = True, how='left')
print(df_left)

            Coal_price    iron  Brent_fut copper_fut  WTI_fut     WTI  \
date                                                                    
2008-12-08       70.45     NaN      43.42   3,294.00    43.71   44.01   
2008-12-09       70.50     NaN      41.53   3,175.50    42.07   42.40   
2008-12-10       71.07     NaN      42.40   3,275.50    43.52   44.09   
2008-12-11       75.88     NaN      47.39   3,287.00    47.98   47.31   
2008-12-12       74.04     NaN      46.41   3,143.50    46.28   46.37   
...                ...     ...        ...        ...      ...     ...   
2022-05-20      417.25  133.66     112.55   9,422.00   113.23  112.04   
2022-05-21      417.25  133.66     112.55   9,422.00   113.23  112.04   
2022-05-22      417.25  133.66     112.55   9,422.00   113.23  111.89   
2022-05-23      416.00  133.99     110.78   9,548.00   110.29  111.23   
2022-05-24      408.00  133.31     110.69   9,453.50   109.77  112.06   

            LNG_fut     bdi  
date                

In [ ]:
df_left['date']= pd.to_datetime(df_left['date'])

df_left = df_left.set_index('date')
df_left

In [13]:
df_left2 = pd.merge(df_left, df_2, left_index = True ,right_index = True, how='left')
print(df_left2)

            Coal_price    iron  Brent_fut copper_fut  WTI_fut     WTI  \
date                                                                    
2008-12-08       70.45     NaN      43.42   3,294.00    43.71   44.01   
2008-12-09       70.50     NaN      41.53   3,175.50    42.07   42.40   
2008-12-10       71.07     NaN      42.40   3,275.50    43.52   44.09   
2008-12-11       75.88     NaN      47.39   3,287.00    47.98   47.31   
2008-12-12       74.04     NaN      46.41   3,143.50    46.28   46.37   
...                ...     ...        ...        ...      ...     ...   
2022-05-20      417.25  133.66     112.55   9,422.00   113.23  112.04   
2022-05-21      417.25  133.66     112.55   9,422.00   113.23  112.04   
2022-05-22      417.25  133.66     112.55   9,422.00   113.23  111.89   
2022-05-23      416.00  133.99     110.78   9,548.00   110.29  111.23   
2022-05-24      408.00  133.31     110.69   9,453.50   109.77  112.06   

            LNG_fut     bdi  USD/AUD(호주)  USD/CAD(

In [17]:
df_left2.columns

Index(['Coal_price', 'iron', 'Brent_fut', 'copper_fut', 'WTI_fut', 'WTI',
       'LNG_fut', 'bdi', 'USD/AUD(호주)', 'USD/CAD(캐나다)', 'USD/COP(콜롬비아)',
       'US 달러지수', 'USD/IDR(인도네시아)', 'USD/RUB(러시아)', 'USD/ZAR(자메이카)', '상하이종합지수',
       '인도지수', '브라질지수', '다우존스지수', '프랑스지수', '영국지수', '독일지수', 'S&P500지수', '항셍지수',
       '나스닥지수', '코스피지수', '니케이225지수', '유로스톡스50지수', '호주지수', '캐나다지수', '베트남GDP',
       '인도GDP', '한국GDP', '일본GDP', '중국GDP', '미국GDP', 'EUGDP', '세계GDP'],
      dtype='object')

In [18]:
df_final= df_left2.drop(['베트남GDP','인도GDP', '한국GDP', '일본GDP', '중국GDP', '미국GDP', 'EUGDP', '세계GDP'],axis=1)

In [20]:
df_final.isnull().sum()

Coal_price          0
iron              686
Brent_fut           0
copper_fut          0
WTI_fut             0
WTI                 0
LNG_fut             0
bdi                 1
USD/AUD(호주)       533
USD/CAD(캐나다)      533
USD/COP(콜롬비아)     533
US 달러지수           533
USD/IDR(인도네시아)    533
USD/RUB(러시아)      533
USD/ZAR(자메이카)     533
상하이종합지수           533
인도지수              533
브라질지수             533
다우존스지수            533
프랑스지수             533
영국지수              533
독일지수              533
S&P500지수          533
항셍지수              533
나스닥지수             533
코스피지수             533
니케이225지수          533
유로스톡스50지수         533
호주지수              533
캐나다지수             533
dtype: int64

In [26]:
df_final = df_final[df_final.index < '2021-01-01']
df_final = df_final[df_final.index >= '2011-01-01']
df_final.isnull().sum()

Coal_price        0
iron              0
Brent_fut         0
copper_fut        0
WTI_fut           0
WTI               0
LNG_fut           0
bdi               0
USD/AUD(호주)       0
USD/CAD(캐나다)      0
USD/COP(콜롬비아)     0
US 달러지수           0
USD/IDR(인도네시아)    0
USD/RUB(러시아)      0
USD/ZAR(자메이카)     0
상하이종합지수           0
인도지수              0
브라질지수             0
다우존스지수            0
프랑스지수             0
영국지수              0
독일지수              0
S&P500지수          0
항셍지수              0
나스닥지수             0
코스피지수             0
니케이225지수          0
유로스톡스50지수         0
호주지수              0
캐나다지수             0
dtype: int64

In [27]:
df_final.to_csv('/content/gdrive/My Drive/Black_coal/data/dataset/project_dataset.csv', index = True, encoding = 'utf-8-sig')